<a href="https://colab.research.google.com/github/jlee335/cs470_Team7_VideoEnhancer/blob/main/Frame_Interpolation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Basic Settings

In [ ]:
#@title Mount google drive
# login with your google account and type authorization code to mount on your google drive.
import os
from google.colab import drive
drive.mount('/gdrive')
 
# Specify the directory path where `Frame-interpolation.ipynb` exists.

root = '/gdrive/My Drive/Frame_Interp/'
root_data = '/gdrive/My Drive/Final_Project/'
bench_dir =  '/gdrive/My Drive/Middlebury_Benchmark/videos'


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


**Import Libraries**

In [ ]:
#@title Import libraries

import os
import numpy as np
import time
from pathlib import Path
from PIL import Image
import cv2 as cv
 
import matplotlib.pyplot as plt
 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import cv2
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.datasets import VOCSegmentation, SBDataset
from torchvision.datasets.vision import StandardTransform
from torchvision.utils import make_grid
from torchvision import transforms
from torchvision.models.vgg import VGG, vgg16, make_layers
from torch.optim import SGD
from torch.utils.tensorboard import SummaryWriter
from math import log10
from torch.autograd import Variable
from easydict import EasyDict as edict
!pip install pydensecrf
from torch.utils.tensorboard import SummaryWriter
import datetime
import pydensecrf.densecrf as dcrf
import pydensecrf.utils as utils

from skimage.measure import compare_ssim
from skimage.measure import compare_psnr
 
!nvidia-smi
!pip install --extra-index-url https://developer.download.nvidia.com/compute/redist nvidia-dali-cuda100

from nvidia.dali.pipeline import Pipeline
import nvidia.dali.ops as ops
import nvidia.dali.types as types

%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import glob


Thu Dec 10 14:33:27 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    25W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#@title Setting parameters
 
torch.manual_seed(470)
torch.cuda.manual_seed(470)
 
args = edict()
 
args.batch_size = 1
args.lr = 1e-4
args.momentum = 0.9
args.weight_decay = 5e-4
args.epoch = 10
args.tensorboard = True
args.gpu = True

 
device = 'cuda' if torch.cuda.is_available() and args.gpu else 'cpu'
 

In [ ]:
#@title Create directory name.
# Create directory name.
result_dir = Path(root) / 'results'
result_dir.mkdir(parents=True, exist_ok=True)

data_dir = Path(root_data) / 'data'
data_dir.mkdir(parents=True, exist_ok=True)

train_dir = data_dir / 'test_data'
valid_dir = data_dir / 'valid_data'

train_dir.mkdir(parents=True, exist_ok=True)
valid_dir.mkdir(parents=True, exist_ok=True)

#@title Set directory
num_trial=0
result_dir= Path(root) / 'results'
parent_dir = result_dir / f'trial_{num_trial}'

# Modify parent_dir here if you want to resume from a , or to rename directory.
# parent_dir = result_dir / 'trial_99'

while parent_dir.is_dir():
    num_trial = int(parent_dir.name.replace('trial_',''))
    parent_dir = result_dir / f'trial_{num_trial+1}'
 
 

In [ ]:
#@title Helper functions

normalize = transforms.Normalize(mean = [0.485, 0.456, 0.406],std = [0.229, 0.224, 0.225])
unnormalize = transforms.Normalize(mean = [-2.118, -2.036, -1.804], std = [4.367, 4.464, 4.444])

def optical_flow_warp(image, image_optical_flow):

    b, _ , h, w = image.size()

    grid = np.meshgrid(range(w), range(h))
    grid = np.stack(grid, axis=-1).astype(np.float64)
    grid[:, :, 0] = grid[:, :, 0] * 2 / (w - 1) -1
    grid[:, :, 1] = grid[:, :, 1] * 2 / (h - 1) -1
    grid = grid.transpose(2, 0, 1)
    grid = np.tile(grid, (b, 1, 1, 1))
    grid = Variable(torch.Tensor(grid))
    if image_optical_flow.is_cuda == True:
        grid = grid.cuda()

    flow_0 = torch.unsqueeze(image_optical_flow[:, 0, :, :] * 31 / (w - 1), dim=1)
    flow_1 = torch.unsqueeze(image_optical_flow[:, 1, :, :] * 31 / (h - 1), dim=1)

    grid = grid + torch.cat((flow_0, flow_1),1)
    grid = grid.transpose(1, 2)
    grid = grid.transpose(3, 2)
    
    output = F.grid_sample(image, grid, padding_mode='border')
    return output

def space_to_depth(x, block_size):
    n, c, h, w = x.size()
    unfolded_x = torch.nn.functional.unfold(x, block_size, stride=block_size)
    return unfolded_x.view(n, c * block_size ** 2, h // block_size, w // block_size)

# Tensorboard Settings


In [ ]:

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

writer = SummaryWriter(logdir)

print()

%load_ext tensorboard
%tensorboard --logdir logs 

Reusing TensorBoard on port 6006 (pid 395), started 0:12:51 ago. (Use '!kill 395' to kill it.)

<IPython.core.display.Javascript object>

# 2. Define Networks

In [ ]:
#@title FlowNet

# https://github.com/amanchadha/FRVSR-GAN/blob/master/FRVSRGAN_Models.py
class ConvLeaky(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(ConvLeaky, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_dim, out_channels=out_dim,
                               kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=out_dim, out_channels=out_dim,
                               kernel_size=3, stride=1, padding=1)
        
        self.conv3 = nn.Conv2d(in_channels=out_dim, out_channels=out_dim,
                               kernel_size=3, stride=1, padding=1)

    def forward(self, input):
        out = self.conv1(input)
        out = F.leaky_relu(out, 0.2)
        out = self.conv2(out)
        out = F.leaky_relu(out, 0.2)
        out = self.conv3(out)
        out = F.leaky_relu(out, 0.2)
        return out


class FNetBlock(nn.Module):
    def __init__(self, in_dim, out_dim, typ):
        super(FNetBlock, self).__init__()
        self.convleaky = ConvLeaky(in_dim, out_dim)
        if typ == "maxpool":
            self.final = lambda x: F.max_pool2d(x, kernel_size=2, ceil_mode=True)
        elif typ == "bilinear":
            self.final = lambda x: F.interpolate(x, scale_factor=2, mode="bilinear")
        else:
            raise Exception('Type does not match any of maxpool or bilinear')

    def forward(self, input):
        out = self.convleaky(input)
        out = self.final(out)
        return out


#@title SR Net
def crop(tensor,top,left,h,w):
    return tensor[:,:,top:top+h,left:left+w]

#https://arxiv.org/pdf/1801.04590.pdf
class IFNet(nn.Module):
    def __init__(self, in_dim=6):
        super(IFNet, self).__init__()
        self.e64 =  FNetBlock(in_dim, 64, typ="maxpool")
        self.e128 =  FNetBlock(64, 128, typ="maxpool")
        self.e256 = FNetBlock(128, 256, typ="maxpool")
        self.e512 = FNetBlock(256, 512, typ="maxpool")
  
        self.d256 = FNetBlock(512, 256, typ="bilinear")
        self.d128  = FNetBlock(256, 128, typ="bilinear")
        self.d64  = FNetBlock(128, 64, typ="bilinear")
        self.d32  = FNetBlock(64, 32, typ="bilinear")

        self.conv1 = nn.Conv2d(in_channels=32, out_channels=3, kernel_size=3, padding=1)
      
        self.three = nn.Sequential(
          nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
          nn.ReLU(),
          nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
          nn.ReLU(),
          nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
          nn.ReLU()
        )
        self.tan = nn.Tanh()
        
    def forward(self, x):

        x = torch.cat(x,dim = 1)
        x = self.e64(x)
        r1 = x
        x = self.e128(x)
        r2 = x
        x = self.e256(x)
        r3 = x
        x = self.e512(x)
        x = self.d256(x)
        x += r3
        x = self.d128(x)
        x += r2
        x = self.d64(x)
        x += r1
        x = self.d32(x)

        x = self.three(x)

        x = self.conv1(x)
        x = self.tan(x)
        
        return x


In [ ]:
#@title Discriminator Net 

class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator,self).__init__()
    self.l_relu = nn.LeakyReLU()
    self.outer_conv = nn.Conv2d(3,64,3)

    self.conv_64c = nn.Conv2d(64,64,3,2)
    self.bn_64c = nn.BatchNorm2d(64)

    self.conv_128e = nn.Conv2d(64,128,3,1)
    self.bn_128e = nn.BatchNorm2d(128)

    self.conv_128c = nn.Conv2d(128,128,3,2)
    self.bn_128c = nn.BatchNorm2d(128)

    self.conv_256e = nn.Conv2d(128,256,3,1)
    self.bn_256e = nn.BatchNorm2d(256)

    self.conv_256c = nn.Conv2d(256,256,3,2)
    self.bn_256c = nn.BatchNorm2d(256)

    self.conv_512e = nn.Conv2d(256,512,3,1)
    self.bn_512e = nn.BatchNorm2d(512)

    self.conv_512c = nn.Conv2d(512,512,3,2)
    self.bn_512c = nn.BatchNorm2d(512)

    self.flatten = nn.AdaptiveAvgPool2d(1)
    self.lin1 = nn.Conv2d(512, 1024, kernel_size=1)
    self.lin2 = nn.Conv2d(1024, 1, kernel_size=1)
    self.sig = nn.Sigmoid()

  def forward(self,x):
    x = self.l_relu(self.outer_conv(x))
    x = self.l_relu(self.bn_64c(self.conv_64c(x)))
    x = self.l_relu(self.bn_128e(self.conv_128e(x)))
    x = self.l_relu(self.bn_128c(self.conv_128c(x)))
    x = self.l_relu(self.bn_256e(self.conv_256e(x)))
    x = self.l_relu(self.bn_256c(self.conv_256c(x)))
    x = self.l_relu(self.bn_512e(self.conv_512e(x)))
    x = self.l_relu(self.bn_512c(self.conv_512c(x)))

    x = self.flatten(x)
    x = self.l_relu(self.lin1(x))
    x = self.sig(self.lin2(x))
    
    return x

In [ ]:
#@title VGG Cutout
class VGG_cutout(nn.Module):
    def __init__(self, original_model):
        super(VGG_cutout, self).__init__()
        self.features = nn.Sequential(nn.Sequential(*list(original_model.children())[0])[:27])
        
    def forward(self, x):
        x = self.features(x)
        return x


In [ ]:
#@title Perceptual loss
# https://github.com/dongheehand/SRGAN-PyTorch/blob/master/losses.py
class perceptual_loss(nn.Module):
    def __init__(self, vgg):
        super(perceptual_loss, self).__init__()
        self.vgg = vgg
        self.criterion = nn.MSELoss()

    def forward(self, HR, SR, layer = 'relu5_4'):
        hr_feat = self.vgg(HR)
        sr_feat = self.vgg(SR)
        
        return self.criterion(hr_feat, sr_feat), hr_feat, sr_feat

NVIDIA DALI dataloader pipeline configuration

In [ ]:
#@title DALI video pipline config
batch_size=3
sequence_length=3

initial_prefetch_size=16

video_directory = os.path.join(train_dir)
valid_directory = os.path.join(bench_dir)

video_files = [video_directory + '/' + f for f in os.listdir(video_directory)]
valid_files = [bench_dir + '/' + f for f in os.listdir(valid_directory)]

shuffle=True
args.batch_size = batch_size

class VideoPipe(Pipeline):
    def __init__(self, batch_size, num_threads, device_id, data, shuffle):
        super(VideoPipe, self).__init__(batch_size, num_threads, device_id, seed=16)
        self.input = ops.VideoReader(device="gpu", filenames=data, sequence_length=sequence_length,
                                     shard_id=0, num_shards=1,
                                     random_shuffle=shuffle, initial_fill=initial_prefetch_size,pad_last_batch = True)

    def define_graph(self):
        output = self.input(name="Reader")
        return output

pipe = VideoPipe(batch_size=batch_size, num_threads=2, device_id=0, data=video_files, shuffle=shuffle)
pipe.build()

valid_pipe = VideoPipe(batch_size=1, num_threads=2, device_id=0, data=valid_files, shuffle=False)
valid_pipe.build()


RuntimeError: ignored

# Training Phase

GAN training Code

In [ ]:
#@title GAN training code


def test_net_valid(generator,epoch_num):
    randcrop = torch.nn.Sequential(
        torchvision.transforms.RandomCrop((320,400), padding=None, pad_if_needed=False, fill=0, padding_mode='constant'),
        )
    generator.eval()

    test_iter = 10 
    
    avg_psnr = 0
    avg_ssim = 0

    for i in range(test_iter):
      pipe_out = valid_pipe.run() 
      pipe_out = pipe_out[0].as_cpu().as_array()
      pipe_out = randcrop(torch.from_numpy(pipe_out).permute(0,1,4,2,3))

      frame_list = torch.chunk(pipe_out,chunks = pipe_out.shape[1],dim=1) 

      prev_frame =  frame_list[0].squeeze(1).to(device).float()/255.0
      frame      =  frame_list[1].squeeze(1).to(device).float()/255.0
      next_frame =  frame_list[2].squeeze(1).to(device).float()/255.0

      orig_img = frame.squeeze(0).transpose(0,2).transpose(0,1).cpu().detach().numpy()

      prev_frame =  normalize(prev_frame).to(device)
      frame      =  normalize(frame).to(device)
      next_frame =  normalize(next_frame).to(device)

      frame_recon = generator((prev_frame,next_frame)) 

      n_image = unnormalize(frame_recon)
      n_image = n_image.squeeze(0).transpose(0,2).transpose(0,1).cpu().detach().numpy()

      psnr = compare_psnr(orig_img,n_image)

     # if i == 0:
        #ssim = compare_ssim(orig_img,n_image,win_size = 11,multichannel=True)
        #avg_ssim = ssim
      avg_psnr += psnr/test_iter
      

    print(f"Validation set testing: psnr = {avg_psnr:.5f}, ssim = {avg_ssim:.5f}")
    writer.add_scalar("PSNR/VALID",  avg_psnr, epoch_num)
    writer.add_scalar("SSIM/VALID",  avg_ssim, epoch_num)

from torch.autograd import Variable

def train_net_GAN(net,discriminator, resume,n_iter,n_iter_desc,gen_epoc,dis_epoch,load_optim = False,fname = []):
    criterion = nn.MSELoss().to(device) 

    randcrop = torch.nn.Sequential(
        torchvision.transforms.RandomCrop(256, padding=None, pad_if_needed=False, fill=0, padding_mode='constant'),
        )

    epoch = 0
    
    vgg_net = torchvision.models.vgg19(pretrained=True).to(device)
    vgg_net_cut = VGG_cutout(vgg_net)
    vgg_net_cut.eval()

    VGG_CUT_LOSS = perceptual_loss(vgg_net_cut)

    optimizer = torch.optim.Adam(net.parameters(),lr=args.lr)
    if load_optim:
      optimizer.load_state_dict(torch.load(fname[0]))
    net.train()
 
    while epoch < gen_epoch: 
        if gen_epoch == 0:
          continue
        t1 = time.time()
        first = True
        loss_total = 0

        for i in range(n_iter):
            pipe_out = pipe.run() 
            pipe_out = pipe_out[0].as_cpu().as_array()
            pipe_out = randcrop(torch.from_numpy(pipe_out).permute(0,1,4,2,3))

            frame_list = torch.chunk(pipe_out,chunks = pipe_out.shape[1],dim=1) 
 
            prev_frame =  frame_list[0].squeeze(1).to(device).float()/255.0
            frame =       frame_list[1].squeeze(1).float()/255.0             
            next_frame =  frame_list[2].squeeze(1).to(device).float()/255.0
 
            prev_frame =  normalize(prev_frame).to(device)
            frame     =   normalize(frame).to(device)
            next_frame =  normalize(next_frame).to(device)
 
            frame_recon = net((prev_frame,next_frame))

            loss,_,_= VGG_CUT_LOSS(frame,frame_recon)
 
            net.zero_grad()
 
            loss.backward()
 
            optimizer.step()
            
            if i % 10 == 0:
              writer.add_scalar("gen-only-loss/train", loss, epoch*n_iter + i)
 
            loss_total += loss.item()/n_iter

        t = time.time() - t1
        
        print("--epoch finished-- " + str(epoch) + " time taken: " + str(t))
        print(f'train loss : {loss_total:.5f}')
        test_net_valid(net,epoch)

        torch.save(net.state_dict(),  result_dir / f'tmp_{type(net).__name__}_.ckpt')
        torch.save(optimizer.state_dict(),  result_dir / f'tmp_{type(optimizer).__name__}_.ckpt')
        epoch += 1

        writer.flush()

    vgg_net = torchvision.models.vgg19(pretrained=True).to(device)
    vgg_net.eval()
    VGG_loss = perceptual_loss(vgg_net)
    cross_ent = nn.BCELoss()
 
    real_label = torch.ones((batch_size, 1,1,1)).to(device)
    fake_label = torch.zeros((batch_size, 1,1,1)).to(device)
 
    ce_loss = nn.BCELoss()
 
    discriminator.train()

    optimizer = torch.optim.Adam(net.parameters(),lr=args.lr * 0.1)
    disc_optimizer = torch.optim.Adam(discriminator.parameters(),lr=args.lr * 0.1)
    if load_optim:
      optimizer.load_state_dict(torch.load(fname[0]))
      disc_optimizer.load_state_dict(torch.load(fname[1]))

    epoch = 0
    torch.save(net.state_dict(),  result_dir / f'disc_{type(net).__name__}_.ckpt')

    print("TRAINING DISCRIMINATOR NETWORK")
    while epoch < dis_epoch: 
        t1 = time.time()
        first = True
        loss_total = 0
        L2_Loss = 0
        adv_Loss = 0
        disc_loss_total = 0
 
        for i in range(n_iter_desc):
            pipe_out = pipe.run()
            pipe_out = pipe_out[0].as_cpu().as_array()
            pipe_out = randcrop(torch.from_numpy(pipe_out).permute(0,1,4,2,3))
 
            frame_list = torch.chunk(pipe_out,chunks = pipe_out.shape[1],dim=1)
 
            prev_frame =  frame_list[0].squeeze(1).to(device).float()/255.0
            frame =       frame_list[1].squeeze(1).float()/255.0              
            next_frame =  frame_list[2].squeeze(1).to(device).float()/255.0
 
            prev_frame =  normalize(prev_frame).to(device)
            frame     =   normalize(frame).to(device)
            next_frame =  normalize(next_frame).to(device)

 
            frame_recon = net((Variable(prev_frame),Variable(next_frame)))

            discriminator.zero_grad() 
            
            fake_prob = discriminator(frame_recon)
            real_prob = discriminator(frame)
            

            fake_error = ce_loss(fake_prob,fake_label) 
            real_error = ce_loss(real_prob,real_label) 
 
            disc_loss = fake_error + real_error

            disc_loss.backward(retain_graph = True)                       

            disc_optimizer.step()                    

            net.zero_grad()                   
 
            percep_loss, hr_feat, sr_feat = VGG_loss((frame + 1.0) / 2.0, (frame_recon + 1.0) / 2.0, layer = 'relu5_4') 
            L2loss,_,_ = VGG_CUT_LOSS(frame,frame_recon) 
 
            percep_loss *= 0.01
            adversarial_loss = cross_ent(discriminator(frame_recon), real_label) * 0.001
 
            total_loss = percep_loss + adversarial_loss + L2loss
 
            total_loss.backward()                         
            
            optimizer.step()                              

            if i % 10 == 0:
              writer.add_scalar("adv-total_loss/train",         total_loss, epoch*n_iter_desc + i)
              writer.add_scalar("adv-adversarial_loss/train",   adversarial_loss, epoch*n_iter_desc + i)
              writer.add_scalar("adv-L2loss/train",             L2loss, epoch*n_iter_desc + i)
              writer.add_scalar("adv-disc_loss/train",          disc_loss, epoch*n_iter_desc + i)
 
            loss_total += total_loss.item()/n_iter_desc
            adv_Loss += adversarial_loss.item()/n_iter_desc
            L2_Loss += L2loss.item()/n_iter_desc
            disc_loss_total += disc_loss.item()/n_iter_desc

        t = time.time() - t1
        
 
        epoch += 1
        torch.save(net.state_dict(),  result_dir / f'tmp_{type(net).__name__}_.ckpt')
        torch.save(discriminator.state_dict(),  result_dir / f'tmp_{type(discriminator).__name__}_.ckpt')
        torch.save(optimizer.state_dict(),  result_dir / f'tmp_{type(optimizer).__name__}_.ckpt')
        torch.save(disc_optimizer.state_dict(),  result_dir / f'tmp_dc_{type(disc_optimizer).__name__}_.ckpt')

        print("--fine grain epoch finished-- " + str(epoch-1) + " time taken: " + str(t))
        print(f'loss : {loss_total:.5f} ,adv_loss :{adv_Loss:.5f},l2_loss: {L2_Loss:.5f} disc_loss :{disc_loss_total:.5f}')
        test_net_valid(net,epoch-1)
        writer.flush()

    torch.save(net.state_dict(),  result_dir / f'disc_{type(net).__name__}_.ckpt')
    torch.save(discriminator.state_dict(),  result_dir / f'disc_{type(discriminator).__name__}_.ckpt')

    torch.save(optimizer.state_dict(),  result_dir / f'disc_{type(optimizer).__name__}_.ckpt')
    torch.save(disc_optimizer.state_dict(),  result_dir / f'disc_dc_{type(disc_optimizer).__name__}_.ckpt')
    
    writer.close()

In [ ]:
#@title Train Model
 
#Set cont as True if you want to continue the training of gernerator based on existing ckpt
#load_disc and load_optimizers are same as cont but for discriminator

#Put generator's file name and discriminator's file name, generator optimization file, discriminator optimization file in disc_file and gen_file, G_optim_file, D_optim_file.
#ex) disc_file = 'tmp_Discriminator_.ckpt'
#ex) gen_file = 'tmp_IFNet_.ckpt'
#ex) G_optim_file = result_dir / 'tmp_Adam_.ckpt'
#ex) D_optim_file = result_dir / 'tmp_dc_Adam_.ckpt'
#Set number of epochs and iterations per epoch

#####################################
cont = True
load_disc = True
load_optimizers = True

disc_file = ''
gen_file = ''
 
G_optim_file = ''
D_optim_file = ''

fname = [G_optim_file,D_optim_file]


gen_epoch = 
gen_iter = 
 
dis_epoch = 
dis_iter = 
####################################### 
 
if cont:
  
  model = IFNet().to(device)
  desc = Discriminator().to(device)
 
 
  model.load_state_dict(torch.load(result_dir / gen_file))
  if load_disc:
    desc.load_state_dict(torch.load(result_dir / disc_file))
 
  train_net_GAN(model,desc,False,gen_iter,dis_iter,gen_epoch,dis_epoch,load_optimizers,fname)
 
else:
  model = IFNet().to(device)
  desc = Discriminator().to(device)
 
  if load_disc:
    desc.load_state_dict(torch.load(result_dir / disc_file))
 
  train_net_GAN(model,desc,False,gen_iter,dis_iter,gen_epoch,dis_epoch,load_optimizers,fname)

# Model Test


In [ ]:
#@title Input video
#Put video file as input in data.
#ex) data='/gdrive/My Drive/Final_Project/data/test_data/Video1.mp4'
#Put where to save your frames of your video.
#We hardly recommend you to MAKE "new foler" for frames.
#DO NOT change part after the path in name.
#ex)  name= '/gdrive/My Drive/Final_Project/data/test_data/frame/'+str(Number)+'.jpg'
###########################
data=''
###################################
vc=cv2.VideoCapture(data)
Number=0

while(True):
  ret, frame=vc.read()
  if not (ret):
    break
  #####################################  
  name= ''+str(Number)+'.jpg'
  #########################################
  cv2.imwrite(name,frame)
  Number+=1

vc.release()
cv2.destroyAllWindows()

In [ ]:

#Set model_file as .ckpt file
# ex) model_file = Path(root)/'Models'/'disc_IFNet_1129.ckpt'

#model_file ='/gdrive/My Drive/Final_Project/results/disc_IFNet_.ckpt'
#You should modify the path mentioned as "HERE".
#The path in "HERE" SHOULD BE SAME!!!!
#ex)change "HERE" as '/gdrive/My Drive/Final_Project/data/test_data/frame'
#DO NOT modify part after "HERE"
#Modify (size) as the size you want.
#(size) SHOULD BE SAME and format of (x,y) and each x and y SHOULD BE DIVIDED TO 8!!!!!
#ex) (800,800)

##########################
model_file =''
##################################
model = IFNet().to(device)
model.load_state_dict(torch.load(result_dir / model_file))
model.eval()

########################
frame_list=os.listdir("HERE")
########################


for i in range(len(frame_list)-1):
  images=[]
  ############################################
  image1= cv2.imread('HERE'+str(i)+'.jpg')
  image1=cv2.resize(image1,(size) )
  image2= cv2.imread('HERE'+str(i+1)+'.jpg')
  image2=cv2.resize(image2,(size) )
  ############################################3
  image1_=image1
  image2_=image2

  loader = transforms.Compose([transforms.ToTensor()])
  image1 = loader(image1).float()
  image1 = image1.unsqueeze(0).to(device)
  images.append(normalize(image1))

  loader = transforms.Compose([transforms.ToTensor()])
  image2 = loader(image2).float()
  image2 = image2.unsqueeze(0).to(device)
  images.append(normalize(image2))

  n_image = model(images)
  n_image = unnormalize(n_image)
 
  n_image2 = n_image.squeeze(0).transpose(0,2).transpose(0,1).cpu().detach().numpy()
  n_image2 =(n_image2*255).astype(np.uint8)

####################################
  svname='HERE'+str(i)+'-1.jpg'
  cv.imwrite(svname,n_image2)
  svname1='HERE'+str(i)+'.jpg'
  cv.imwrite(svname1,image1_)
  svname2='HERE'+str(i+1)+'.jpg'
####################################
  cv.imwrite(svname2,image2_)


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


In [ ]:
#in VideoWriter, you should put code where to save your result video file.
#ex) video=cv2.VideoWriter('/gdrive/My Drive/VideoCapture/capture/Hi.avi',fourcc,15, (800,600))

#fourccis avi file's codec. Divx, Xvid, H.264 is available if you set your video file as .avi.
#If you want your output as .mp4 change fourcc 'XVID' as'mp4v' and change your video file name as .mp4.
#You can change FPS of video as you want.

#########################
fourcc = cv2.VideoWriter_fourcc(*'XVID')
FPS=
video=cv2.VideoWriter('/gdrive/My Drive/Final_Project/data/test_data/result/Result.avi',fourcc,FPS, (800,800))

n=os.listdir('HERE')

###################################

def number_from_filename (fname):
  num = fname.split('.')[0]
  num2 = fname.split('-')
  if len(num2) != 1:
    num = float(num2[0]) + 0.5
  else:
    num = float(num)
  return num

n = sorted(n, key = lambda x : number_from_filename(x))


print(n)
for i in range (len(n)):
  ####################################3
  img=cv2.imread('HERE'+n[i])
  ###############################
  video.write(img)

video.release()
cv2.destroyAllWindows()